In [1]:
import os, re
import pandas as pd
import preprocessor as p
!pip install --disable-pip-version-check -q pyspellchecker

In [2]:
from spellchecker import SpellChecker
spellchecker = SpellChecker(language="en")
spellchecker.correction("frustated")

'frustrated'

In [3]:
# Setup preprocessor
p.set_options(p.OPT.MENTION, p.OPT.URL, p.OPT.HASHTAG, p.OPT.EMOJI, p.OPT.SMILEY)
print(p.clean("RT @test: Foobar #awesome https://www.baz.me 22/7 is pi yup!"))
print(p.clean("@test Foobar #awesome https://www.baz.me 22/7 is pi yup!"))

def clean(word):
    if word[:2] == "RT":
        word = p.clean(word[3:])[2:]
    else:
        word = p.clean(word)
    word = re.sub(r"[^\sA-Za-z]", "", word)
    # return re.sub(r"\s+", " ", word)
    return " ".join(map(spellchecker.correction, re.sub(r"\s+", " ", word).lower().split()))

# Foobar becomes forbear, spellchecker autoconverts to lowercase
print(clean("RT @test: Foobar #awesome https://www.baz.me 22/7 is pi yup!"))
print(clean("@test Foobar #awesome https://www.baz.me 22/7 is pi yup!"))

# For apostrophes, we decided to combine the abbreviated words
print(clean("I've done this  before @@xoxo"))

RT : Foobar 22/7 is pi yup!
Foobar 22/7 is pi yup!
forbear is pi yup
forbear is pi yup
ive done this before


In [4]:
train_data, test_data, valid_data = [], [], []
size_table = {"train": [], "test": [], "valid": []}

emoji_map = pd.read_csv(os.getcwd() + "\Datasets\Original\emoji_map_1791.csv")
emoji_idx = [1381, 1424, 1392, 1447, 186, 1389, 1420, 1620, 1403, 763, 1138, 1446]

In [5]:
emoji_list = list(map(lambda x: emoji_map.iloc[x, 0], emoji_idx))
emoji_list

['😂', '😭', '😍', '🙄', '❤', '😊', '😩', '🤔', '😘', '🏽', '💯', '🙃']

In [6]:
# This particular emoji has a different behaviour
print(emoji_map.iloc[emoji_idx[-3], 0])

🏽


In [7]:
for num in range(1, 13):
    train_df = pd.read_csv(f"https://raw.githubusercontent.com/RussellDash332/CS3244-Twemoji/main/Datasets/train_text_emoji_{num}.csv")
    test_df = pd.read_csv(f"https://raw.githubusercontent.com/RussellDash332/CS3244-Twemoji/main/Datasets/test_text_emoji_{num}.csv")
    valid_df = pd.read_csv(f"https://raw.githubusercontent.com/RussellDash332/CS3244-Twemoji/main/Datasets/valid_text_emoji_{num}.csv")

    def list_to_emoji(string):
        emojis = list(map(int, string[1:-1].split(",")))
        return str().join(map(lambda x: emoji_map.iloc[x, 0], emojis))

    def emoji_to_index(emoji):
        return emoji_list.index(emoji)

    # Convert "[1381, 1424]" to "😂😭"
    train_df["annotations"] = train_df["annotations"].apply(list_to_emoji)
    test_df["annotations"] = test_df["annotations"].apply(list_to_emoji)
    valid_df["annotations"] = valid_df["annotations"].apply(list_to_emoji)

    # Update: drop the rows with multiple emojis and keep the single-labelled ones
    train_df = train_df[train_df.annotations == emoji_list[num-1]]
    test_df = test_df[test_df.annotations == emoji_list[num-1]]
    valid_df = valid_df[valid_df.annotations == emoji_list[num-1]]
    
    # Convert "😂" to 0
    train_df["labels"] = train_df["annotations"].apply(emoji_to_index)
    test_df["labels"] = test_df["annotations"].apply(emoji_to_index)
    valid_df["labels"] = valid_df["annotations"].apply(emoji_to_index)

    # Apply cleaning from twitter-preprocessor
    train_df["tweets"] = train_df["tweets"].apply(clean)
    test_df["tweets"] = test_df["tweets"].apply(clean)
    valid_df["tweets"] = valid_df["tweets"].apply(clean)

    train_data.append(train_df)
    test_data.append(test_df)
    valid_data.append(valid_df)

    # print(f"Train data size from emoji {num}:", train_df.shape)
    # print(f"Test data size from emoji {num}:", test_df.shape)
    # print(f"Valid data size from emoji {num}:", valid_df.shape)

    size_table["train"].append(train_df.shape[0])
    size_table["test"].append(test_df.shape[0])
    size_table["valid"].append(valid_df.shape[0])

In [8]:
size_table = pd.DataFrame(size_table, index=list(range(1, 13)))
size_table["sum"] = size_table.train + size_table.test + size_table.valid
size_table

# training is balanced, good to go
# stick with this t-t-v combination
# this was before duplicate removal

,train,test,valid,sum
1,4504,1804,1782,8090
2,4240,1570,1429,7239
3,4324,1093,1172,6589
4,4609,1215,1228,7052
5,4232,731,556,5519
6,4567,743,749,6059
7,4345,1308,807,6460
8,4557,1216,1840,7613
9,4086,512,591,5189
10,4289,1724,1715,7728


In [9]:
size_table.apply(sum)

train    52888
test     13486
valid    13481
sum      79855
dtype: int64

In [10]:
train_data[0]

,id,annotations,tweets,labels
0,743419925738496000,😂,school is so dead o my god,0
1,742411940677492736,😂,lol ive been told mine is worse than yours but...,0
2,744394777974628352,😂,im excited to hear them that shit is going rat...,0
3,743679858199298049,😂,damn alicia knows everything even indirect tweets,0
4,742579985588887552,😂,that sound like everything,0
...,...,...,...,...
4995,744391978171904000,😂,always,0
4996,747994577357930497,😂,had to flex on tia people,0
4997,746430898464161792,😂,do eco is really the coolest do,0
4998,742758236986576897,😂,we be so jobless,0


In [11]:
train_data = pd.concat(train_data)
test_data = pd.concat(test_data)
valid_data = pd.concat(valid_data)

print("Before missing and duplicate removal: ", train_data.shape, test_data.shape, valid_data.shape)

Before missing and duplicate removal:  (52888, 4) (13486, 4) (13481, 4)


In [12]:
train_data = train_data[~train_data.duplicated()]
train_data = train_data [train_data.tweets != ""]

test_data = test_data[~test_data.duplicated()]
test_data = test_data[test_data.tweets != ""]

valid_data = valid_data[~valid_data.duplicated()]
valid_data = valid_data[valid_data.tweets != ""]

print("After missing and duplicate removal: ", train_data.shape, test_data.shape, valid_data.shape)

After missing and duplicate removal:  (52884, 4) (13485, 4) (13481, 4)


In [13]:
train_data = train_data.drop("id", axis=1)
test_data = test_data.drop("id", axis=1)
valid_data = valid_data.drop("id", axis=1)

In [14]:
train_data.sample(n=10, random_state=1010)

,annotations,tweets,labels
759,😍,i appreciate all you little booty women in thi...,2
3861,😩,currently having a music shoot on snapchat lol,6
794,❤,miss you guys so much man,4
1555,💯,just received his first box hit him up for an ...,10
3497,💯,i keep telling you nights,10
651,😩,some jamba juice sounds sooo good right now,6
15,❤,my heart,4
1261,🙃,tired all day but wide awake at night,11
3486,🤔,i gotta question,7
2249,💯,whether you doing good or bad somebody is goin...,10


In [15]:
train_data.to_csv(os.getcwd() + "\Datasets\\train_text_emoji_clean.csv", index=False)
test_data.to_csv(os.getcwd() + "\Datasets\\test_text_emoji_clean.csv", index=False)
valid_data.to_csv(os.getcwd() + "\Datasets\\valid_text_emoji_clean.csv", index=False)

- XLNet -> sentiment analysis / Text Classification (https://www.topbots.com/leading-nlp-language-models-2020/#language-models-2020-3)
- OpenAI’s GPT2 -> supervised learning on task-specific datasets (https://insights.daffodilsw.com/blog/top-5-nlp-language-models)
- Embedding Layers (https://machinelearningmastery.com/what-are-word-embeddings/)
- Word2Vec (https://machinelearningmastery.com/what-are-word-embeddings/)\
- GloVe (https://machinelearningmastery.com/what-are-word-embeddings/)
- Parsing (https://www.analyticsvidhya.com/blog/2020/12/understanding-text-classification-in-nlp-with-movie-review-example-example/#h2_6)
- Semantic (https://www.analyticsvidhya.com/blog/2020/12/understanding-text-classification-in-nlp-with-movie-review-example-example/#h2_7)